In [1]:
import os
import requests
from getpass import getpass
import tempfile

# 临时文件存储
cookiejar = tempfile.NamedTemporaryFile(delete=False)
netrc_file = tempfile.NamedTemporaryFile(delete=False)

def prompt_credentials():
    """提示用户输入Earthdata Login的用户名和密码，并将其写入到netrc文件"""
    print("Enter your Earthdata Login or other provider supplied credentials")
    username = input("Username (bigvan): ") or "bigvan"
    password = getpass("Password: ")

    with open(netrc_file.name, 'w') as netrc:
        netrc.write(f"machine urs.earthdata.nasa.gov login {username} password {password}\n")
    
    print(f"Credentials stored in {netrc_file.name}")

def exit_with_error(message):
    """输出错误信息并退出"""
    print("\nUnable to Retrieve Data\n")
    print(message)
    print("\nhttps://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ASTGTM.003/ASTGTMV003_N03E008_dem.tif")
    exit(1)

def detect_app_approval():
    """检测应用授权状态"""
    url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ASTGTM.003/ASTGTMV003_N03E008_dem.tif'
    try:
        response = requests.get(url, cookies=cookiejar, allow_redirects=True, auth=(netrc_file.name, ''))
        if response.status_code not in [200, 301, 302]:
            exit_with_error("Please ensure that you have authorized the remote application by visiting the link below.")
    except requests.RequestException as e:
        exit_with_error(f"Request failed: {e}")

def setup_auth():
    """设置认证逻辑"""
    url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ASTGTM.003/ASTGTMV003_N03E008_dem.tif'
    response = requests.head(url)
    
    # 如果返回状态不是200或者304，表示需要URS认证
    if response.status_code not in [200, 304]:
        detect_app_approval()

def fetch_urls(url_list):
    """下载指定的URL列表中的数据"""
    for url in url_list:
        # 获取文件名
        filename = os.path.basename(url)
        stripped_filename = filename.split('?')[0]  # 去掉URL中的查询参数

        # 下载文件
        try:
            response = requests.get(url, cookies=cookiejar, auth=(netrc_file.name, ''), stream=True)
            print(response)
            if response.status_code == 200:
                with open(stripped_filename, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                print(f"Downloaded: {stripped_filename}")
            else:
                exit_with_error("Command failed with error. Please retrieve the data manually.")
        except requests.RequestException as e:
            exit_with_error(f"Request failed: {e}")




In [2]:
def main():
    prompt_credentials()
    setup_auth()

    # 可以从文件或其他来源提供下载链接
    url_list = [
        'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/ASTGTM.003/ASTGTMV003_N03E008_dem.tif'
    ]

    fetch_urls(url_list)

if __name__ == '__main__':
    main()

Enter your Earthdata Login or other provider supplied credentials
Credentials stored in C:\Users\bigvan\AppData\Local\Temp\tmpdi9i9hcq
